In [1]:
import torch 
from datasets import Dataset
from peft import get_peft_model,LoraConfig
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM,Trainer,TrainingArguments,DataCollatorForSeq2Seq,BitsAndBytesConfig
print(torch.cuda.is_available())

c:\KM\Virtual ENV\venv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [ ]:
import numpy as np
import pandas as pd

sam_train=pd.read_csv('Data/samsum-train.csv')
sam_train=sam_train.head(7500)
sam_train

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
4995,13828654,"Thomas: Sir, we have a problem.\r\nMarshall: W...",Thomas is informing Marshall that Broderick Sp...
4996,13728621,"Rachel: Hi, how are you doing?\r\nCharlie: goo...",Charlie is in New York. Rachel will see Charli...
4997,13680705,Estrella: But I will try to get some job in li...,Estrella is going to try and get a job in Lisb...
4998,13728136,Zoe: Hiii\r\nSammy: NO\r\nZoe: what?\r\nSammy:...,Too general; no specific info.


In [3]:
sam_train.columns=['id','text','summary']
sam_train=sam_train.iloc[:,1:]
sam_train

,text,summary
0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...
4995,"Thomas: Sir, we have a problem.\r\nMarshall: W...",Thomas is informing Marshall that Broderick Sp...
4996,"Rachel: Hi, how are you doing?\r\nCharlie: goo...",Charlie is in New York. Rachel will see Charli...
4997,Estrella: But I will try to get some job in li...,Estrella is going to try and get a job in Lisb...
4998,Zoe: Hiii\r\nSammy: NO\r\nZoe: what?\r\nSammy:...,Too general; no specific info.


In [4]:
ds=Dataset.from_pandas(df=sam_train)
split=ds.train_test_split(test_size=0.2,seed=42)
train_df=split['train']
test_df=split['test']

In [5]:
MODEL='google/flan-t5-base'
tokenizer=AutoTokenizer.from_pretrained(MODEL,use_fast=True)
bnb_config=BitsAndBytesConfig(load_in_8bit=True)
model=AutoModelForSeq2SeqLM.from_pretrained(MODEL,quantization_config=bnb_config,device_map='auto')

In [6]:
in_len=512
out_len=128

def preprocess_batch(ex):
    instruction=f'Summarise:{ex["text"]}'
    inp_tok=tokenizer(instruction,truncation=True,max_length=in_len)
    target_summary=ex['summary']+tokenizer.eos_token
    out_tok=tokenizer(text_target=target_summary,truncation=True,max_length=out_len)
    labels=[(tok if tok!=tokenizer.pad_token_id else -100) for tok in out_tok['input_ids']]
    return {'input_ids':inp_tok['input_ids'],'attention_mask':inp_tok['attention_mask'],'labels':labels}


In [7]:
tokenized_train=train_df.map(preprocess_batch,remove_columns=train_df.column_names)
tokenized_test=test_df.map(preprocess_batch,remove_columns=test_df.column_names)
tokenized_train.set_format(type='torch')
tokenized_test.set_format(type='torch')


Map: 100%|██████████| 1000/1000 [00:00<00:00, 2043.06 examples/s]


In [8]:
lora_config=LoraConfig(
    r=16,lora_alpha=32,lora_dropout=0.05,bias="none",task_type='SEQ_2_SEQ_LM',target_modules=["q","v"]
)
model=get_peft_model(model,lora_config)
model.print_trainable_parameters()


trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096


In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="./flan_t5_lora_out",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_accumulation_steps=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=20,
    eval_strategy="no",
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,

)

C:\Users\KARAN MATTOO\AppData\Local\Temp\ipykernel_21256\4060611941.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

c:\KM\Virtual ENV\venv2\Lib\site-packages\transformers\data\data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
20,12.017900
40,10.402200
60,7.382800
80,12.881800
100,8.338700
120,6.676500
140,5.940200
160,8.146600
180,7.089000
200,7.011600


TrainOutput(global_step=1500, training_loss=8.601260864257812, metrics={'train_runtime': 1371.9083, 'train_samples_per_second': 8.747, 'train_steps_per_second': 1.093, 'total_flos': 3378924243929088.0, 'train_loss': 8.601260864257812, 'epoch': 3.0})

In [11]:
# --- 1. Imports and Setup ---
import torch
import evaluate
from datasets import Dataset

# Assuming 'model' and 'tokenizer' are already loaded with LoRA adapters
# If you re-started your notebook, reload your model like this:
# from peft import PeftModel
# model = PeftModel.from_pretrained(model, "./flan_t5_lora_final") 
# model = model.to('cuda') # Ensure the model is on the GPU

# Use a batch size of 1 to minimize VRAM usage during generation
SAFE_BATCH_SIZE = 4
# We use the original test_df dataframe (from Cell 3/4)
test_data_samples = test_df 

# --- 2. Manual Inference Loop ---

predictions = []
reference_summaries = []

print(f"Starting memory-safe inference on {len(test_data_samples)} samples (Batch Size: {SAFE_BATCH_SIZE})...")

# Loop through the test data, processing one sample (or SAFE_BATCH_SIZE samples) at a time
for i in range(0, len(test_data_samples), SAFE_BATCH_SIZE):
    
    # Select the current batch chunk from the dataset
    batch_chunk = test_data_samples.select(range(i, min(i + SAFE_BATCH_SIZE, len(test_data_samples))))
    
    # Format the input using your instruction tuning prefix
    input_texts = [f'Summarize:{ex["text"]}' for ex in batch_chunk]
    
    # Tokenize the batch and move to the GPU
    # NOTE: Padding=True is essential here, even for batch size 1, 
    # as it ensures the input tensor is correctly structured.
    input_batch = tokenizer(
        input_texts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=512 # Match your new max_in length
    ).to(model.device)
    
    # === GENERATION (THE MEMORY-INTENSIVE PART) ===
    with torch.no_grad(): # Disable gradient calculation for inference
        outputs = model.generate(
            **input_batch,
            max_length=64, # Your desired output length
            num_beams=4,   # Use beam search for higher quality
            do_sample=False
        )
    
    # Decode outputs and append
    decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    predictions.extend(decoded_preds)
    reference_summaries.extend(batch_chunk['summary'])
    
    # Optional: Print progress
    if (i // SAFE_BATCH_SIZE) % 50 == 0:
        print(f"Processed {i + SAFE_BATCH_SIZE} samples...")

# --- 3. Final ROUGE Scoring ---

rouge = evaluate.load("rouge")

# Post-processing function to clean text for accurate ROUGE scoring
def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

clean_preds, clean_labels = postprocess_text(predictions, reference_summaries)

# Calculate the final metrics
final_rouge_results = rouge.compute(
    predictions=clean_preds, 
    references=clean_labels, 
    use_stemmer=True
)

print("\n--- Final ROUGE Results (Memory-Safe) ---")
print(final_rouge_results)

Starting memory-safe inference on 1000 samples (Batch Size: 4)...
Processed 4 samples...
Processed 204 samples...
Processed 404 samples...
Processed 604 samples...
Processed 804 samples...

--- Final ROUGE Results (Memory-Safe) ---
{'rouge1': np.float64(0.45351973443715454), 'rouge2': np.float64(0.2125908276921321), 'rougeL': np.float64(0.3736699292557541), 'rougeLsum': np.float64(0.3733172262480615)}
